# 开始使用

## 结合poetry使用Jupyter

在poetry中支持jupyter和ipykernel：

In [ ]:
!poetry add ipykernel jupyter

创建一个名为`langchani-book-poetry-env`的可用内核：

In [ ]:
!poetry run python -m ipykernel install --user --name=langchani-book-py3.10-ipykernel

## 加载环境变量

In [2]:
import os
from dotenv import load_dotenv, find_dotenv

import sys
import io
original_stdout = sys.stdout 
sys.stdout = io.StringIO()
load_dotenv(find_dotenv(), override=True)
sys.stdout = original_stdout

## 使用langchain

## 使用langserve

## 访问langserve服务

<div class="alert alert-success">
<b>架构解偶：</b>在python中访问远程langserve的机制，有助于将GPT服务和应用模块在架构上解耦。
</div>

例如，要结合AI生成能力创建文档，下面这个封装会非常有用：

In [3]:
debug = True
base_url = "http://localhost:8000"

from IPython.display import display, Markdown
def print_content(data):
    for char in data:
        print(char, end="", flush=True)

from langserve import RemoteRunnable
def remote_runnable(runnable, params = [], base_url = base_url, debug = debug):
    chain = RemoteRunnable(f"{base_url}/{runnable}")
    def ask(*args):
        kwargs = dict(zip(params, args))
        if debug:
            print_content(chain.stream(kwargs))
        else:
            return chain.invoke(kwargs)
    return ask

或者打包为一个python文件，直接引用：

In [5]:
from Utils import *

RUNNABLE_BASE_URL:  http://localhost:8000


### GPT

In [6]:
gpt35 = remote_runnable("langserve/gpt35", ["question"])
gpt4 = remote_runnable("langserve/gpt4", ["question"])

In [7]:
gpt35("我的父母结婚时为什么没有邀请我？")

可能有很多原因导致父母结婚时没有邀请你。也许他们认为你还太小，或者他们想要一个私人的、小型的婚礼。也有可能是因为他们的婚礼计划是在你不在场的时候进行的。无论原因是什么，重要的是理解并接受他们的决定，并且祝福他们未来的幸福生活。

In [8]:
gpt4("我的父母结婚时为什么没有邀请我？")

如果你的父母在结婚时没有邀请你，可能有几个原因：

1. 你还没有出生：这是最常见的原因。如果你的父母在你出生之前结婚，那么你自然不可能被邀请。

2. 私密或小型婚礼：有些夫妇可能选择举行一个非常私密的婚礼，只邀请非常亲近的家人或朋友，或者甚至只有两个人单独举行仪式。

3. 地理或物理障碍：可能在你的父母结婚时，你因为某些原因（如年幼、学业、工作或其他责任）无法到场。

4. 家庭关系：有时由于家庭内部关系复杂，可能会导致某些家庭成员没有被邀请参加婚礼。

5. 误解或沟通问题：也有可能是由于沟通不畅或误解导致你没有收到邀请。

6. 其他个人原因：每个家庭的情况都是独特的，可能有其他你不知道的原因。

如果这个问题让你感到困惑或不安，与你的父母进行开放和诚实的对话可能会帮助你了解当时的情况和他们的决定。通常，这种情况是由于你还未出生或非常小，所以不可能参与他们的婚礼。

### 通义千问

In [6]:
tongyi = remote_runnable("langserve/tongyi", ["question"])

In [7]:
tongyi("你是什么模型?")

我是阿里云开发的大规模语言模型，我叫通义千问。

In [87]:
tongyi("我的父母结婚时为什么没有邀请我?")

因为您在他们结婚的时候还没有出生，所以他们自然无法邀请您参加他们的婚礼。通常情况下，只有已经出生并能够理解事件意义的人才会被邀请参加婚礼。

### Chatglm3-6B

In [95]:
chatglm6b = remote_runnable("langserve/chatglm6b", ["question"])

In [96]:
chatglm6b("你是什么模型?")


 我是一个名为 ChatGLM3-6B 的人工智能助手，是基于清华大学KEG 实验室和智谱 AI公司于 2023 年共同训练的语言模型开发的。我的任务是针对用户的问题和要求提供适当的答复和支持。

In [88]:
chatglm6b("我的父母结婚时为什么没有邀请我?")


 作为人工智能助手，我无法了解您个人生活中的具体情况。但是，通常情况下，婚礼是一种庆祝两个人成为夫妇的仪式，因此，新婚夫妇很少会邀请孩子作为嘉宾。在某些情况下，他们可能会邀请孩子参加婚礼，但这取决于孩子的年龄和婚礼的规模。如果您希望了解更多关于您父母婚礼的信息，您可以尝试与他们沟通，了解他们当时的想法和决定。

### lambda函数

自定义的lambda函数一般不访问大模型，而是以较快速度在本地计算，因此不需要支持流式打印，所以无需封装。

In [5]:
chain = RemoteRunnable(f"{base_url}/langserve/add")
chain.invoke({"x": 1, "y": 2})

'The result is: 3'

In [6]:
chain = RemoteRunnable(f"{base_url}/langserve/add_one")
chain.invoke(42)

'The result is: 43'

## langchain的核心类

### RunnableLambda

#### 使用invoke和batch

In [85]:
from langchain_core.runnables import RunnableLambda
import time

# 定义你的函数
def hello(name):
    return f"hi {name} ! Welcome To Langchain!"

# 创建一个MyRunnableLambda实例
runnable = RunnableLambda(hello)

# 使用invoke获得一次性输出
print(runnable.invoke("Alice"))  # 输出 "hi Alice !"

# 注意：因为hello函数没有流式输出，因此stream也是没有流式输出效果的
for output in runnable.stream("Alice"):
    time.sleep(0.3)
    print(output)  # 输出 "hi Alice !"

hi Alice ! Welcome To Langchain!
hi Alice ! Welcome To Langchain!


#### 使用batch获得批量结果

In [89]:
# batch中会自动调用invoke
runnable.batch(["Alice", "Bob"])

['hi Alice ! Welcome To Langchain!', 'hi Bob ! Welcome To Langchain!']

#### 获得stream的流式输出效果

In [90]:
from langchain_core.runnables import RunnableLambda
import time

# 定义你的函数
def hello(name):
    yield "hi "
    yield name
    yield "! "
    yield "Welcome"
    yield " to"
    yield " Langchain!"

# 创建一个MyRunnableLambda实例
runnable = RunnableLambda(hello)

# invoke会等到所有结果获得后合并到一起输出
print(runnable.invoke("Alice"))  # 输出 "hi Alice !"

# 因为hello函数实现了yield，stream就会有流式输出效果
for output in runnable.stream("Alice"):
    time.sleep(0.3)
    print(output, end = "")  # 输出 "hi Alice !"

hi Alice! Welcome to Langchain!
hi Alice! Welcome to Langchain!

#### 使用 @chain 达到同样的效果

In [91]:
from langchain_core.runnables import chain
import time

# 定义你的函数
@chain
def hello(name):
    yield "hi "
    yield name
    yield "! "
    yield "Welcome"
    yield " to"
    yield " Langchain!"

print(hello.invoke("Alice"))  # 输出 "hi Alice !"

for output in hello.stream("Alice"):
    time.sleep(0.3)
    print(output, end = "")  # 输出 "hi Alice !"

hi Alice! Welcome to Langchain!
hi Alice! Welcome to Langchain!